In [ ]:
import pandas as pd
import numpy as np
import duckdb 
import pydantic 
import dataclasses
import sqlalchemy

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [ ]:
sp_dt = pd.read_json('/home/garcia-ln/Documentos/real-state-prices/data/raw/sp_properties.json')
rj_dt = pd.read_json('/home/garcia-ln/Documentos/real-state-prices/data/raw/rj_properties.json')
pa_dt = pd.read_json('/home/garcia-ln/Documentos/real-state-prices/data/raw/pa_properties.json')
bh_dt = pd.read_json('/home/garcia-ln/Documentos/real-state-prices/data/raw/bh_properties.json')

# JSON > Parquet
Before starting the process of cleaning and transforming the data for  our analysis, i'm gonna make sure to define all data types and convert the files into ```.parquet``` format so that i'm always dealing with optimazed performance DataFrames, no matter the situation.  

For data, i'm gonna check the dtypes and and other basic info about each file:  

But before getting into that, i need to add some information to each file, so that the final DataSet has all the information need it for the analysis and model development. 

In [ ]:
dfs = [sp_dt, rj_dt, pa_dt, bh_dt]
cities = ['Sao_Paulo', 'Rio_de_Janeiro', 'Porto_Alegre', 'Belo_Horizonte']

dfs[0]['city'], dfs[1]['city'], dfs[2]['city'], dfs[3]['city'] = cities[0], cities[1], cities[2], cities[3]
for city in dfs:
    display(city)
    display(city.info())
    display(city.describe())

In [ ]:
def set_dtype(self):
    self['type'] = self['type'].astype('category')
    self['address'] = self['address'].astype('category')
    self['neighborhood'] = self['neighborhood'].astype('category')
    self['footage'] = self['footage'].astype('int16')
    self['doorms'] = self['doorms'].astype('int8')
    self['garages'] = self['garages'].astype('int8')
    self['city'] = self['city'].astype('category')
    self['price'] = self['price'].astype('float64')
    return self

for city in dfs:
    set_dtype(city)
    display(city.describe())
    display(city.info())
    display(city)


In [ ]:
sp_dt.to_parquet('~/Documentos/real-state-prices/data/processed/sp_properties.parquet', index=None)
rj_dt.to_parquet('~/Documentos/real-state-prices/data/processed/rj_properties.parquet', index=None)
pa_dt.to_parquet('~/Documentos/real-state-prices/data/processed/pa_properties.parquet', index=None)
bh_dt.to_parquet('~/Documentos/real-state-prices/data/processed/bh_properties.parquet', index=None)

In [ ]:
# con = duckdb.connect()

results = con.execute(
    "CREATE TABLE sp_tbl AS SELECT * FROM read_parquet('~/Documentos/real-state-prices/data/processed/sp_properties.parquet')"
).df()